## Mask R-CNN - Inspect Trained Model

Code and visualizations to test, debug, and evaluate the Mask R-CNN model.

In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline 


# mlcc
from samples.mlcc import mlcc3

config = mlcc3.MLCCConfig()
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to Shapes trained weights
MODEL_PATH = os.path.join(ROOT_DIR, "logs/mlcc220200403T1219/mask_rcnn_mlcc2_0080.h5")

Using TensorFlow backend.


## Configurations

In [2]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        1024
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE             

## Notebook Preferences

In [3]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [4]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load Validation Dataset

In [5]:
# Build validation dataset
dataset = mlcc3.MLCCDataset()
dataset.load_mlcc('../../mlcc2', 'test')

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!
Images: 65
Classes: ['BG', 'Fault', 'MLCC']


## Load Model

In [6]:
MODEL_DIR

'/docker/git/Mask_RCNN/logs'

In [7]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [8]:
model.load_weights(MODEL_PATH, by_name=True)







Re-starting from epoch 80


## Run Detection

In [ ]:
# Draw only faults
folder = "./mlcc2_test_res2"
if not os.path.exists(folder):
    os.makedirs(folder)
#image_id = random.choice(dataset.image_ids)
for i in range(len(dataset.image_ids)):
    name = os.path.join(folder, "mlcc2-{:03d}.png".format(i+8))
    image_id = i
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
    info = dataset.image_info[image_id]
    print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                           dataset.image_reference(image_id)))
    # Run object detection
    results = model.detect([image], verbose=1)

    fault_idx = np.where(results[0]['class_ids']==1)[0]
    res_fault = {}
    res_fault['rois'] = results[0]['rois'][fault_idx]
    res_fault['class_ids'] = results[0]['class_ids'][fault_idx]
    res_fault['masks'] = results[0]['masks'][:,:,fault_idx]
    res_fault['scores'] = results[0]['scores'][fault_idx]

    # Display results
    ax = get_ax(1)
    r = res_fault
    visualize.save_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                dataset.class_names, r['scores'], ax=ax,
                                title="Predictions",
                             path=name)
    #log("gt_class_id", gt_class_id)
    #log("gt_bbox", gt_bbox)
    #log("gt_mask", gt_mask)

image ID: mlcc.0 (0) 
Processing 1 images
image                    shape: (512, 512, 3)         min:    0.00000  max:  149.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min: -123.70000  max:   25.30000  float64
image_metas              shape: (1, 15)               min:    0.00000  max:  512.00000  int64
anchors                  shape: (1, 65472, 4)         min:   -0.04428  max:    0.99494  float32
image ID: mlcc.1 (1) 
Processing 1 images
image                    shape: (512, 512, 3)         min:    0.00000  max:  144.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min: -123.70000  max:   20.30000  float64
image_metas              shape: (1, 15)               min:    0.00000  max:  512.00000  int64
anchors                  shape: (1, 65472, 4)         min:   -0.04428  max:    0.99494  float32
image ID: mlcc.2 (2) 
Processing 1 images
image                    shape: (512, 512, 3)         min:    0.00000  max:  134.00000  uint8
molded_images       

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  



*** No instances to display *** 

image ID: mlcc.21 (21) 
Processing 1 images
image                    shape: (512, 512, 3)         min:    0.00000  max:  134.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min: -123.70000  max:   10.30000  float64
image_metas              shape: (1, 15)               min:    0.00000  max:  512.00000  int64
anchors                  shape: (1, 65472, 4)         min:   -0.04428  max:    0.99494  float32


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


image ID: mlcc.22 (22) 
Processing 1 images
image                    shape: (512, 512, 3)         min:    0.00000  max:  112.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min: -123.70000  max:  -11.70000  float64
image_metas              shape: (1, 15)               min:    0.00000  max:  512.00000  int64
anchors                  shape: (1, 65472, 4)         min:   -0.04428  max:    0.99494  float32


...

image ID: mlcc.43 (43) 
Processing 1 images
image                    shape: (512, 512, 3)         min:    0.00000  max:   78.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min: -123.70000  max:  -38.90000  float64
image_metas              shape: (1, 15)               min:    0.00000  max:  512.00000  int64
anchors                  shape: (1, 65472, 4)         min:   -0.04428  max:    0.99494  float32


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  



*** No instances to display *** 

image ID: mlcc.44 (44) 
Processing 1 images
image                    shape: (512, 512, 3)         min:    0.00000  max:  138.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min: -123.70000  max:   14.30000  float64
image_metas              shape: (1, 15)               min:    0.00000  max:  512.00000  int64
anchors                  shape: (1, 65472, 4)         min:   -0.04428  max:    0.99494  float32


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


image ID: mlcc.45 (45) 
Processing 1 images
image                    shape: (512, 512, 3)         min:    0.00000  max:   71.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min: -123.70000  max:  -50.90000  float64
image_metas              shape: (1, 15)               min:    0.00000  max:  512.00000  int64
anchors                  shape: (1, 65472, 4)         min:   -0.04428  max:    0.99494  float32


In [ ]:
#image_id = random.choice(dataset.image_ids)
image_id = 57
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
info = dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       dataset.image_reference(image_id)))
# Run object detection
results = model.detect([image], verbose=1)

# Display results
ax = get_ax(1)
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'], ax=ax,
                            title="Predictions")
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

In [ ]:
def acc_cal(gt, res):
    unique_gt, count_gt = np.unique(gt, return_counts=True)
    unique_res, count_res = np.unique(res, return_counts=True)
    
    if len(count_gt) == 2:
        if len(count_res) == 2:
            acc_mlcc = count_res[1]/count_gt[1]
            acc_fault = count_res[0]/count_gt[0]
        else:
            acc_mlcc = count_res[0]/count_gt[1]
            acc_fault = None
    else:
        acc_mlcc = count_res[0]/count_gt[0]
        acc_fault = None
    
    return [acc_mlcc, acc_fault]

In [ ]:
accuracy = []

for idx in dataset.image_ids:
    image_id = idx

    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
    info = dataset.image_info[image_id]
    results = model.detect([image], verbose=1)
    
    acc = acc_cal(gt_class_id, results[0]['class_ids'])
    accuracy.append(acc)

In [ ]:
ac_res = np.array(accuracy)

In [ ]:
fault_acc = []
for i in range(ac_res.shape[0]):
    if ac_res[i][1] != None:
        fault_acc.append(ac_res[i][1])

In [ ]:
mlcc_acc = ac_res[:,0]

In [ ]:
np.mean(mlcc_acc)

In [ ]:
np.mean(fault_acc)

### Precision-Recall

In [ ]:
# Draw precision-recall curve
AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                          r['rois'], r['class_ids'], r['scores'], r['masks'])
visualize.plot_precision_recall(AP, precisions, recalls)

### Compute mAP @ IoU=50 on Batch of Images

In [ ]:
# Compute VOC-style Average Precision
def compute_batch_ap(image_ids):
    APs = []
    for image_id in image_ids:
        # Load image
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset, config,
                                   image_id, use_mini_mask=False)
        # Run object detection
        results = model.detect([image], verbose=0)
        # Compute AP
        r = results[0]
        AP, precisions, recalls, overlaps =\
            utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                              r['rois'], r['class_ids'], r['scores'], r['masks'])
        APs.append(AP)
    return APs


# Pick a set of random images
image_ids = np.random.choice(dataset.image_ids, 10)
APs = compute_batch_ap(image_ids)
print("mAP @ IoU=50: ", np.mean(APs))

In [ ]:
def compute_batch_conf(image_ids):
    confs = []
    for image_id in image_ids:
        # Load image
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset, config,
                                   image_id, use_mini_mask=False)
        # Run object detection
        results = model.detect([image], verbose=0)
        # Compute AP
        r = results[0]
        AP, precisions, recalls, overlaps =\
            utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                              r['rois'], r['class_ids'], r['scores'], r['masks'])
        confs.append([AP, precisions, recalls, overlaps])
    return confs

results = compute_batch_conf(dataset.image_ids)

In [ ]:
results = np.array(results)

In [ ]:
results[0][2]